# Testing your app

Once your app is running you can query it. You can simply do that by navigating to the URL that ngrok gave you above or through code in a different python session. For example, once the frontend app is running, you can navigate to:
http://YOUR_SERVER_DOMAIN/search?query=hello+world where YOUR_SERVER_DOMAIN is something like XXXX-XX-XX-XX-XX.ngrok.io, which is printed above in Colab or that is your external IP on GCP.

The code below shows how to issue a query from python. This is also how our testing code will issue queries to your search engine, so make sure to test your search engine this way after you deploy it to GCP and before submission. Command line instructions for deploying your search engine to GCP are available at `run_frontend_in_gcp.sh`. Note that we will not only issue training queries to your search engine, but also test queries, i.e. queries that you've never seen before.

In [2]:
import json

with open('queries_train.json', 'rt') as f:
  queries = json.load(f)

In [5]:
def average_precision_at_40(true_list, predicted_list, k=40):
    true_set = frozenset(true_list)
    predicted_list = predicted_list[:k]
    precisions = []
    for i,doc_id in enumerate(predicted_list):        
        if doc_id in true_set:
            prec = (len(precisions)+1) / (i+1)            
            precisions.append(prec)
    if len(precisions) == 0:
        return 0.0
    return round(sum(precisions)/len(precisions),3)

def average_precision(true_list, predicted_list):
    true_set = frozenset(true_list)
    precisions = []
    for i,doc_id in enumerate(predicted_list):        
        if doc_id in true_set:
            prec = (len(precisions)+1) / (i+1)            
            precisions.append(prec)
    if len(precisions) == 0:
        return 0.0
    return round(sum(precisions)/len(precisions),3)

def recall(true_list, predicted_list):
    true_set = frozenset(true_list)
    rec = 0
    for i in predicted_list:
      if i in true_set:
        rec += 1
    return rec/len(true_set)

In [26]:
import requests
from time import time

url = 'http://35.225.7.222:8080'

qs_res = []
for q, true_wids in queries.items():
  duration, ap = None, None
  t_start = time()
  try:
    res = requests.get(url + '/search', {'query': q}, timeout=35)
    duration = time() - t_start
    if res.status_code == 200:
      pred_wids, _ = zip(*res.json())
      apa40 = average_precision_at_40(true_wids, pred_wids)
      ap = average_precision(true_wids, pred_wids)
      r = recall(true_wids, pred_wids)
  except:
    pass
  print(f"{q} -> {len(pred_wids)}")
  
  qs_res.append((q, duration, apa40, ap, r))

print(qs_res)
sum_duration = 0
sum_apa40 = 0
sum_ap = 0
sum_r = 0
for tup in qs_res:
  sum_duration += tup[1]
  sum_apa40 += tup[2]
  sum_ap += tup[3]
  sum_r += tup[4]

avg_duration = sum_duration/len(qs_res)
avg_apa40 = sum_apa40/len(qs_res)
avg_ap = sum_ap/len(qs_res)
avg_r = sum_r/len(qs_res)

print(f"average recall is {avg_r}")
print(f"average precision is {avg_ap}")
print(f"average precision at 40 is {avg_apa40}")
print(f"average time is {avg_duration}")

best marvel movie -> 4
How do kids come to world? -> 14
Information retrieval -> 14
LinkedIn -> 7
How to make coffee? -> 4
Ritalin -> 4
How to make wine at home? -> 16
Most expensive city in the world -> 15
India -> 27
how to make money fast? -> 8
Netflix -> 18
Apple computer -> 6
The Simpsons -> 4
World cup -> 4
How to lose weight? -> 17
Java -> 5
Air Jordan -> 8
how to deal with depression? -> 12
How do you make gold -> 7
Marijuana -> 4
How to make hummus -> 10
Winter -> 31
Rick and Morty -> 14
Natural Language processing -> 9
World Cup 2022 -> 14
Dolly the sheep -> 6
Cigarettes -> 4
What is the best place to live in? -> 16
Elon musk -> 13
How do you breed flowers? -> 8
[('best marvel movie', 2.219882011413574, 0.5, 0.5, 0.022222222222222223), ('How do kids come to world?', 2.740659713745117, 0.127, 0.127, 0.04), ('Information retrieval', 1.0459835529327393, 0.979, 0.979, 0.3), ('LinkedIn', 0.10203075408935547, 1.0, 1.0, 0.4666666666666667), ('How to make coffee?', 1.434638261795044,

In [9]:
try:
  page_rank = requests.post(url + '/get_pagerank', json=[22634890, 23154189, 306742])
  print(page_rank.json())
  page_view = requests.post(url + '/get_pageview', json=[22634890, 23154189, 306742])
  print(page_view.json())
except:
  print("some error")

[5.485321527478058, 0.8112487603091939, 4.720355599766869]
[558, 475, 4641]
